In [6]:
import json
import math

In [7]:
f = open("compressed_tree.json")
tree = json.loads(f.read())
layers = tree["layers"]
classes = tree["classes"]
f.close()

In [8]:
field_width = {
	"src": 16,
	"dst": 16,
	"protocl": 8,
}

In [9]:
def worst_case(tree):

	rmt = []

	step = 0

	for layer in layers:

		# assume that each range requires all of 2*k bits when performing prefix expansion
		num_ranges = len(layers[layer])
		range_expansion_bits = 2 * field_width[layer]

		tcam = {
			"id": f"{layer}_range",
			"step": step,
			"match": "ternary",
			"entries": num_ranges,
			"key_size": range_expansion_bits
		}

		# assume no pointer reuse for metadata storage
		ram = {
			"id": f"{layer}_meta",
			"step": step,
			"match": "exact",
			"method": "index",
			"key_size": math.ceil(math.log2(num_ranges)),
			"data_size": len(classes)
		}

		rmt.append(tcam)
		rmt.append(ram)

		step += 1

	return rmt

worst_case_rmt = worst_case(tree)
f = open("worst_case_rmt.json", "w+")
f.write(json.dumps(worst_case_rmt, indent=4))
f.close()


In [10]:
! command python3 ideal-rmt-simulator/sim.py worst_case_rmt.json

TCAM mapping: 
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SRAM mapping: 
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
id mapping: 
[['dst_range', 'dst_meta'], ['src_range', 'src_meta'], ['protocl_range', 'protocl_meta'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
